In [ ]:
#!pip install pycryptodome
#!pip install PySimpleGUI
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA3_256
import Crypto.Random
import numpy as np
import scipy as sp
import imageio
import PySimpleGUI as sg


#randCount = 0
#My TRNG function
def TRNG(N):
  #global randCount
  #randCount= randCount+1
  #print(randCount)
  image = imageio.imread(values['-TRNG-'])
  image_1d = image.ravel()
  image_1d = image_1d[image_1d != 0] #Usun zera z wejscia
  tc = 0

  array8bits=[]
  processedNumbers=[]
  #Lista liczb zwracanych
  counter = 0
  size = 0
  for x in image_1d:
      tc = tc + 1
      if(tc > 4000):
          array8bits.append(x%2)
          counter = counter + 1
          if(counter == 8):
              out = 0
              for bit in array8bits:
                  out = (out << 1) | bit
              array8bits = []
              processedNumbers.append(out)
              size +=1
              if(size==N):
                  return bytes(processedNumbers)
              counter = 0
  return bytes(processedNumbers)


def generateKeys():
  #Generate both keys and store them in files
  key = RSA.generate(2048)
  private_key = key.export_key(randfunc=TRNG)
  with open("private.pem", "wb") as f:
      f.write(private_key)

  public_key = key.publickey().export_key()
  with open("public.pem", "wb") as f:
      f.write(public_key)

def generateSignature(privateKeyPath, inputFilePath):
  #Sign the thing  with private key
  key = RSA.import_key(open(privateKeyPath).read())
  #Read input file
  with open(inputFilePath,"rb") as f:
    myFile=f.read()
  h= SHA3_256.new(myFile)
  signature = pkcs1_15.new(key).sign(h)
  with open("signature.txt", "wb") as f:
    f.write(signature)

def authData(publicKeyPath, inputFilePath, signature):
  #Read input file
  with open(inputFilePath,"rb") as f:
    myFile=f.read()
  #Authenticate the thing and say if it was tampered with
  key = RSA.import_key(open(publicKeyPath).read())
  h= SHA3_256.new(myFile)
  with open(signature, "rb") as f:
    signature = f.read()
  try:
      pkcs1_15.new(key).verify(h, signature)
      return "The signature is valid."
  except (ValueError, TypeError):
      return "The signature is not valid."

layout = [
            [sg.Text('Photo for TRNG', size=(12,1)), sg.Input(key='-TRNG-'), sg.FileBrowse()],
            [sg.Text('Private Key', size=(12,1)), sg.Input(key='-PRIVK-'), sg.FileBrowse()],
            [sg.Text('Public Key', size=(12,1)), sg.Input(key='-PUBK-'), sg.FileBrowse()],
            [sg.Text('Input File', size=(12,1)), sg.Input(key='-IN-'), sg.FileBrowse()],
            [sg.Text('Signature', size=(12,1)), sg.Input(key='-SIG-'), sg.FileBrowse()],
            [sg.Button('Gen Keys'), sg.Button('Gen Signature'), sg.Button('Authenticate')] ]

#randCount= 0
window = sg.Window('RSA Auth Program', layout)

while True:             # Event Loop
    event, values = window.read()
    print(event, values)
    if event in (None, 'Exit'):
        break
    if event =='Gen Keys':
      generateKeys()
      #print(randCount)
      sg.popup_quick_message('Keys Generated!', text_color='white', background_color='red')
    if event == 'Gen Signature':
      generateSignature(values['-PRIVK-'],values['-IN-'])
      sg.popup_quick_message('Signature Generated!', text_color='white', background_color='red')
    if event == 'Authenticate':
      out = authData(values['-PUBK-'],values['-IN-'],values['-SIG-'])
      sg.popup_quick_message('Result: ',out,text_color='white', background_color='red')

window.close()

Gen Keys {'-TRNG-': '/home/lab/Downloads/52591441-psychedelic-rainbow-spiral-background.png', 'Browse': '/home/lab/Downloads/52591441-psychedelic-rainbow-spiral-background.png', '-PRIVK-': '', 'Browse0': '', '-PUBK-': '', 'Browse1': '', '-IN-': '', 'Browse2': '', '-SIG-': '', 'Browse3': ''}
Gen Signature {'-TRNG-': '/home/lab/Downloads/52591441-psychedelic-rainbow-spiral-background.png', 'Browse': '/home/lab/Downloads/52591441-psychedelic-rainbow-spiral-background.png', '-PRIVK-': '/home/lab/Downloads/private.pem', 'Browse0': '/home/lab/Downloads/private.pem', '-PUBK-': '/home/lab/Downloads/public.pem', 'Browse1': '/home/lab/Downloads/public.pem', '-IN-': '/home/lab/Downloads/flower.jpg', 'Browse2': '/home/lab/Downloads/flower.jpg', '-SIG-': '', 'Browse3': ''}
Authenticate {'-TRNG-': '/home/lab/Downloads/52591441-psychedelic-rainbow-spiral-background.png', 'Browse': '/home/lab/Downloads/52591441-psychedelic-rainbow-spiral-background.png', '-PRIVK-': '/home/lab/Downloads/private.pem', '